# Gold standard curation: Preprocessing and single-step regression

In this stage of gold standard curation, we will do the data preprocessing, selection, and single-step regression for the 153 traits in our question set. This file shows the reference steps using the trait "Breast Cancer" as an example. The workflow consists of the following steps:

1. Preprocess all the cohorts related to this trait. Each cohort should be converted to a tabular form and saved to a csv file, with columns being genetic factors, the trait, and age, gender if available;
2. If there exists at least one cohort with age or gender information, conduct regression analysis with genetic features together with age or gender as the regressors.


# 1. Basic setup

In [2]:
import os

# Set your preferred name
USER = "Shuyi"
# Set the data and output directories
DATA_ROOT = '/content/drive/MyDrive/AI4Science_Public/data'
OUTPUT_ROOT = '/content/drive/MyDrive/AI4Science_Public/output1'
TRAIT = "Endometrioid Cancer"

OUTPUT_DIR = os.path.join(OUTPUT_ROOT, USER, '-'.join(TRAIT.split()))
JSON_PATH = os.path.join(OUTPUT_DIR, "cohort_info.json")
if not os.path.exists(OUTPUT_DIR):
    os.makedirs(OUTPUT_DIR, exist_ok=True)

# Gene symbol normalization may take 1-2 minutes. You may set it to False for debugging.
NORMALIZE_GENE = True

In [1]:
# This cell is only for use on Google Colab. Skip it if you run your code in other environments

import os
from google.colab import drive

drive.mount('/content/drive', force_remount=True)
proj_dir = '/content/drive/MyDrive/AI4Science_Public'
os.chdir(proj_dir)

Mounted at /content/drive


In [3]:
import sys
sys.path.append('/content/drive/MyDrive/Colab Notebooks')
!pip install mygene
!pip install sparse_lmm

# 2. Data preprocessing and selection

## 2.1. The TCGA Xena dataset

In TCGA Xena, there is either zero or one cohort related to the trait. We search the names of subdirectories to see if any matches the trait. If a match is found, we directly obtain the file paths.

In [4]:
import os
from utils import *

dataset = 'TCGA'
dataset_dir = os.path.join(DATA_ROOT, dataset)
os.listdir(dataset_dir)[:10]

['.DS_Store',
 'TCGA_Esophageal_Cancer_(ESCA)',
 'TCGA_Pheochromocytoma_Paraganglioma_(PCPG)',
 'TCGA_Bile_Duct_Cancer_(CHOL)',
 'TCGA_Kidney_Chromophobe_(KICH)',
 'TCGA_Thyroid_Cancer_(THCA)',
 'TCGA_Sarcoma_(SARC)',
 'TCGA_Adrenocortical_Cancer_(ACC)',
 'TCGA_Thymoma_(THYM)',
 'TCGA_Glioblastoma_(GBM)']

If no match is found, jump directly to GEO in Part 2.2

In [ ]:
trait_subdir = "TCGA_Endometrioid_Cancer_(UCEC)"
cohort = 'Xena'
# All the cancer traits in Xena are binary
trait_type = 'binary'
# Once a relevant cohort is found in Xena, we can generally assume the gene and clinical data are available
is_available = True

clinical_data_file = os.path.join(dataset_dir, trait_subdir, 'TCGA.UCEC.sampleMap_UCEC_clinicalMatrix')
genetic_data_file = os.path.join(dataset_dir, trait_subdir, 'TCGA.UCEC.sampleMap_HiSeqV2_PANCAN.gz')

In [ ]:
import pandas as pd

clinical_data = pd.read_csv(clinical_data_file, sep='\t', index_col=0)
genetic_data = pd.read_csv(genetic_data_file, compression='gzip', sep='\t', index_col=0)
age_col = gender_col = None

In [ ]:
_, clinical_data_cols = check_rows_and_columns(clinical_data)

In [ ]:
clinical_data_cols[:10]

['CDE_ID_3226963',
 '_INTEGRATION',
 '_PANCAN_CNA_PANCAN_K8',
 '_PANCAN_Cluster_Cluster_PANCAN',
 '_PANCAN_DNAMethyl_PANCAN',
 '_PANCAN_DNAMethyl_UCEC',
 '_PANCAN_RPPA_PANCAN_K8',
 '_PANCAN_UNC_RNAseq_PANCAN_K16',
 '_PANCAN_miRNA_PANCAN',
 '_PANCAN_mirna_UCEC']

Read all the column names in the clinical dataset, to find the columns that record information about age or gender.
Reference prompt:

In [ ]:
f'''
Below is a list of column names from a biomedical dataset. Please examine it and identify the columns that are likely to contain information about patients' age. Additionally, please do the same for columns that may hold data on patients' gender. Please provide your answer by strictly following this format, without redundant words:
candidate_age_cols = [col_name1, col_name2, ...]
candidate_gender_cols = [col_name1, col_name2, ...]
If no columns match a criterion, please provide an empty list.

Column names:
{clinical_data_cols}
'''

"\nBelow is a list of column names from a biomedical dataset. Please examine it and identify the columns that are likely to contain information about patients' age. Additionally, please do the same for columns that may hold data on patients' gender. Please provide your answer by strictly following this format, without redundant words:\ncandidate_age_cols = [col_name1, col_name2, ...]\ncandidate_gender_cols = [col_name1, col_name2, ...]\nIf no columns match a criterion, please provide an empty list.\n\nColumn names:\n['CDE_ID_3226963', '_INTEGRATION', '_PANCAN_CNA_PANCAN_K8', '_PANCAN_Cluster_Cluster_PANCAN', '_PANCAN_DNAMethyl_PANCAN', '_PANCAN_DNAMethyl_UCEC', '_PANCAN_RPPA_PANCAN_K8', '_PANCAN_UNC_RNAseq_PANCAN_K16', '_PANCAN_miRNA_PANCAN', '_PANCAN_mirna_UCEC', '_PANCAN_mutation_PANCAN', '_PATIENT', '_cohort', '_primary_disease', '_primary_site', 'additional_pharmaceutical_therapy', 'additional_radiation_therapy', 'additional_surgery_locoregional_procedure', 'additional_treatment_co

In [ ]:
candidate_age_cols = ['age_at_initial_pathologic_diagnosis', 'days_to_birth', 'days_to_collection', 'days_to_death', 'days_to_initial_pathologic_diagnosis', 'days_to_last_followup', 'days_to_last_known_alive', 'days_to_new_tumor_event_additional_surgery_procedure', 'days_to_new_tumor_event_after_initial_treatment', 'year_of_initial_pathologic_diagnosis']

candidate_gender_cols = ['gender']

Choose a single column from the candidate columns that record age and gender information respectively.
If no column meets the requirement, keep 'age_col' or 'gender_col' to None

In [ ]:
preview_df(clinical_data[candidate_age_cols])

{'age_at_initial_pathologic_diagnosis': [59.0, 54.0, 69.0, 51.0, 67.0],
 'days_to_birth': [nan, -19818.0, -25518.0, -18785.0, -24477.0],
 'days_to_collection': [957.0, 90.0, 811.0, 103.0, 5015.0],
 'days_to_death': [nan, nan, nan, nan, 3251.0],
 'days_to_initial_pathologic_diagnosis': [0.0, 0.0, 0.0, 0.0, 0.0],
 'days_to_last_followup': [1249.0, 13.0, 1202.0, 313.0, nan],
 'days_to_last_known_alive': [nan, nan, nan, nan, nan],
 'days_to_new_tumor_event_additional_surgery_procedure': [nan,
  nan,
  517.0,
  nan,
  nan],
 'days_to_new_tumor_event_after_initial_treatment': [396.0,
  nan,
  517.0,
  nan,
  nan],
 'year_of_initial_pathologic_diagnosis': [2011.0,
  2013.0,
  2011.0,
  2013.0,
  1996.0]}

In [ ]:
age_col = 'age_at_initial_pathologic_diagnosis'

In [ ]:
preview_df(clinical_data[candidate_gender_cols])

{'gender': ['FEMALE', 'FEMALE', 'FEMALE', 'FEMALE', 'FEMALE']}

In [ ]:
gender_col = 'gender'

In [ ]:
selected_clinical_data = xena_select_clinical_features(clinical_data, TRAIT, age_col, gender_col)

In [ ]:
if NORMALIZE_GENE:
    genetic_data = normalize_gene_symbols_in_index(genetic_data)

INFO:biothings.client:querying 1-1000...
INFO:biothings.client:done.
INFO:biothings.client:Finished.
INFO:biothings.client:Pass "returnall=True" to return complete lists of duplicate or missing query terms.
INFO:biothings.client:querying 1-1000...
INFO:biothings.client:done.
INFO:biothings.client:Finished.
INFO:biothings.client:Pass "returnall=True" to return complete lists of duplicate or missing query terms.
INFO:biothings.client:querying 1-1000...
INFO:biothings.client:done.
INFO:biothings.client:Finished.
INFO:biothings.client:Pass "returnall=True" to return complete lists of duplicate or missing query terms.
INFO:biothings.client:querying 1-1000...
INFO:biothings.client:done.
INFO:biothings.client:Finished.
INFO:biothings.client:Pass "returnall=True" to return complete lists of duplicate or missing query terms.
INFO:biothings.client:querying 1-1000...
INFO:biothings.client:done.
INFO:biothings.client:Finished.
INFO:biothings.client:Pass "returnall=True" to return complete lists of

In [ ]:
merged_data = selected_clinical_data.join(genetic_data.T).dropna()
merged_data.head()

,Endometrioid Cancer,Age,Gender,ARHGEF10L,HIF3A,RNF17,RNF10,RNF11,RNF13,GTF2IP1,...,SLC7A10,PLA2G2C,TULP2,NPY5R,GNGT2,GNGT1,TULP3,BCL6B,GSTK1,SELP
sampleID,,,,,,,,,,,,,,,,,,,,,
TCGA-2E-A9G8-01,1,59.0,0.0,-1.175592,0.864474,-0.531035,0.426928,-0.601278,-1.67421,0.080406,...,2.149614,-0.086682,-0.748878,-0.987117,-0.262433,4.06601,0.333923,-1.501827,0.998505,-2.315833
TCGA-4E-A92E-01,1,54.0,0.0,0.215608,-0.178726,7.416765,0.115128,-0.378178,-0.71591,1.207206,...,-2.090786,-0.086682,-0.207478,-1.587117,-0.143633,0.93371,0.174223,-0.877027,0.217905,0.162267
TCGA-5B-A90C-01,1,69.0,0.0,-0.764492,-2.620626,-0.531035,-0.360672,-0.411278,-0.38501,0.412606,...,-1.380786,-0.086682,1.194822,-0.407417,-1.269133,0.77641,0.573223,-0.710827,-0.555895,-0.691533
TCGA-5S-A9Q8-01,1,51.0,0.0,-0.907792,-3.403426,-0.531035,-0.019372,-1.538578,-1.41421,0.358106,...,-0.637486,1.357518,-0.748878,-1.587117,-1.495533,1.98251,0.406423,-1.374527,0.759805,-1.819733
TCGA-A5-A1OH-01,1,86.0,0.0,-1.823192,5.243674,-0.531035,-0.365172,-0.476078,-0.72361,0.165706,...,6.647914,-0.086682,0.330522,0.758183,0.961667,3.00791,0.208623,0.311373,-1.538495,-0.985533


In [ ]:
print(f"The merged dataset contains {len(merged_data)} samples.")

The merged dataset contains 187 samples.


In [ ]:
is_trait_biased, merge_data = judge_and_remove_biased_features(merged_data, TRAIT, trait_type=trait_type)
is_trait_biased

For the feature 'Endometrioid Cancer', the least common label is '0' with 13 occurrences. This represents 6.95% of the dataset.
The distribution of the feature 'Endometrioid Cancer' in this dataset is fine.

Quartiles for 'Age':
  25%: 57.0
  50% (Median): 65.0
  75%: 73.0
Min: 33.0
Max: 90.0
The distribution of the feature 'Age' in this dataset is fine.

For the feature 'Gender', the least common label is '0.0' with 187 occurrences. This represents 100.00% of the dataset.
The distribution of the feature 'Gender' in this dataset is severely biased.



False

In [ ]:
merged_data.head()
if not is_trait_biased:
    merge_data.to_csv(os.path.join(OUTPUT_DIR, cohort+'.csv'), index=False)

In [ ]:
save_cohort_info(cohort, JSON_PATH, is_available, is_trait_biased, merged_data)

A new JSON file was created at: /content/drive/MyDrive/AI4Science_Public/output1/Shuyi/Endometrioid-Cancer/cohort_info.json


## 2.2. The GEO dataset

In GEO, there may be one or multiple cohorts for a trait. Each cohort is identified by an accession number. We iterate over all accession numbers in the corresponding subdirectory, preprocess the cohort data, and save them to csv files.

In [ ]:
dataset = 'GEO'
trait_subdir = "Endometrioid-Cancer"

trait_path = os.path.join(DATA_ROOT, dataset, trait_subdir)
os.listdir(trait_path)

['GSE94524',
 'GSE108838',
 'GSE109059',
 'GSE226823',
 'GSE109058',
 'GSE155311',
 'GSE25405',
 'GSE120490',
 'GSE94523',
 'GSE155761']

Repeat the below steps for all the accession numbers

In [ ]:
cohort = accession_num = "GSE155761"
cohort_dir = os.path.join(trait_path, accession_num)
soft_file, matrix_file = get_relevant_filepaths(cohort_dir)
soft_file, matrix_file

('/content/drive/MyDrive/AI4Science_Public/data/GEO/Endometrioid-Cancer/GSE155761/GSE155761_family.soft.gz',
 '/content/drive/MyDrive/AI4Science_Public/data/GEO/Endometrioid-Cancer/GSE155761/GSE155761-GPL21145_series_matrix.txt.gz')

### Inital filtering and clinical data preprocessing

In [ ]:
background_prefixes = ['!Series_title', '!Series_summary', '!Series_overall_design']
clinical_prefixes = ['!Sample_geo_accession', '!Sample_characteristics_ch1']

background_info, clinical_data = get_background_and_clinical_data(matrix_file, background_prefixes, clinical_prefixes)
print(background_info)

!Series_title	"The methylomic landscape of fallopian tube lesions associated with ovarian high-grade serous carcinoma"
!Series_summary	"This SuperSeries is composed of the SubSeries listed below."
!Series_overall_design	"Refer to individual Series"


In [ ]:
clinical_data.head()

,!Sample_geo_accession,GSM4698868,GSM4698869,GSM4698870,GSM4698871,GSM4698872,GSM4698873,GSM4698874,GSM4698875,GSM4698876,...,GSM4698891,GSM4698892,GSM4698893,GSM4698894,GSM4698895,GSM4698896,GSM4698897,GSM4698898,GSM4698899,GSM4698900
0,!Sample_characteristics_ch1,uniqueid: S5,uniqueid: N5,uniqueid: S1,uniqueid: N1,uniqueid: S6,uniqueid: N6,uniqueid: S2,uniqueid: N2,uniqueid: S7,...,uniqueid: P11,uniqueid: S11,uniqueid: N13,uniqueid: L13,uniqueid: NA,uniqueid: NA,uniqueid: NA,uniqueid: NA,uniqueid: NA,uniqueid: NA
1,!Sample_characteristics_ch1,histology: Serous Tubal Intraepithelial Carcin...,histology: Adjacent Normal Fallopian Tube Epit...,histology: Serous Tubal Intraepithelial Carcin...,histology: Adjacent Normal Fallopian Tube Epit...,histology: Serous Tubal Intraepithelial Carcin...,histology: Adjacent Normal Fallopian Tube Epit...,histology: Serous Tubal Intraepithelial Carcin...,histology: Adjacent Normal Fallopian Tube Epit...,histology: Serous Tubal Intraepithelial Carcin...,...,histology: P53 Signature Lesion,histology: Serous Tubal Intraepithelial Carcin...,histology: Adjacent Normal Fallopian Tube Epit...,histology: Dormant Serous Tubal Intraepithelia...,histology: NA,histology: NA,histology: NA,histology: NA,histology: NA,histology: NA
2,!Sample_characteristics_ch1,tissue type: FFPE tissue,tissue type: FFPE tissue,tissue type: FFPE tissue,tissue type: FFPE tissue,tissue type: FFPE tissue,tissue type: FFPE tissue,tissue type: FFPE tissue,tissue type: FFPE tissue,tissue type: FFPE tissue,...,tissue type: FFPE tissue,tissue type: FFPE tissue,tissue type: FFPE tissue,tissue type: FFPE tissue,tissue type: FFPE tissue,tissue type: FFPE tissue,tissue type: FFPE tissue,tissue type: FFPE tissue,tissue type: FFPE tissue,tissue type: FFPE tissue
3,!Sample_characteristics_ch1,patient age: unknown,patient age: unknown,patient age: 66,patient age: 66,patient age: 75,patient age: 75,patient age: 50,patient age: 50,patient age: 71,...,patient age: 56,patient age: 56,patient age: 70,patient age: 70,patient age: NA,patient age: NA,patient age: NA,patient age: NA,patient age: NA,patient age: NA
4,!Sample_characteristics_ch1,race: W,race: W,race: W,race: W,race: W,race: W,race: W,race: W,race: W,...,race: W,race: W,race: W,race: W,race: NA,race: NA,race: NA,race: NA,race: NA,race: NA


In [ ]:
clinical_data_unique = get_unique_values_by_row(clinical_data)
clinical_data_unique

{0: ['uniqueid: S5',
  'uniqueid: N5',
  'uniqueid: S1',
  'uniqueid: N1',
  'uniqueid: S6',
  'uniqueid: N6',
  'uniqueid: S2',
  'uniqueid: N2',
  'uniqueid: S7',
  'uniqueid: N7',
  'uniqueid: S3',
  'uniqueid: N3',
  'uniqueid: S8',
  'uniqueid: N8',
  'uniqueid: S4',
  'uniqueid: N4',
  'uniqueid: T6',
  'uniqueid: T8',
  'uniqueid: N9',
  'uniqueid: S9',
  'uniqueid: N12',
  'uniqueid: S12',
  'uniqueid: N11',
  'uniqueid: P11',
  'uniqueid: S11',
  'uniqueid: N13',
  'uniqueid: L13',
  'uniqueid: NA'],
 1: ['histology: Serous Tubal Intraepithelial Carcinoma (STIC)',
  'histology: Adjacent Normal Fallopian Tube Epithelia',
  'histology: Concomitant High Grade Serous Carcinoma Tumor',
  'histology: P53 Signature Lesion',
  'histology: Dormant Serous Tubal Intraepithelial Carcinoma',
  'histology: NA'],
 2: ['tissue type: FFPE tissue'],
 3: ['patient age: unknown',
  'patient age: 66',
  'patient age: 75',
  'patient age: 50',
  'patient age: 71',
  'patient age: 79',
  'patient ag

Analyze the metadata to determine data relevance and find ways to extract the clinical data.
Reference prompt:

In [ ]:
f'''As a biomedical research team, we are selecting datasets to study the association between the human trait \'{TRAIT}\' and genetic factors, optionally considering the influence of age and gender. After searching the GEO database and parsing the matrix file of a series, we obtained background information and sample characteristics data. We will provide textual information about the dataset background, and a Python dictionary storing a list of unique values for each field of the sample characteristics data. Please carefully review the provided information and answer the following questions about this dataset:
1. Does this dataset contain gene expression data? (Note: Pure miRNA data is not suitable.)
2. For each of the traits \'{TRAIT}\', 'age', and 'gender', please address these points:
   (1) Is there human data available for this trait?
   (2) If so, identify the key in the sample characteristics dictionary where unique values of this trait is recorded. The key is an integer. The trait information might be explicitly recorded, or can be inferred from the field with some biomedical knowledge or understanding about the data collection process.
   (3) Choose an appropriate data type (either 'continuous' or 'binary') for each trait. Write a Python function to convert any given value of the trait to this data type. The function should handle inference about the trait value and convert unknown values to None.
   Name the functions 'convert_trait', 'convert_age', and 'convert_gender', respectively.

Background information about the dataset:
{background_info}

Sample characteristics dictionary (from "!Sample_characteristics_ch1", converted to a Python dictionary that stores the unique values for each field):
{clinical_data_unique}
'''

'As a biomedical research team, we are selecting datasets to study the association between the human trait \'Endometrioid Cancer\' and genetic factors, optionally considering the influence of age and gender. After searching the GEO database and parsing the matrix file of a series, we obtained background information and sample characteristics data. We will provide textual information about the dataset background, and a Python dictionary storing a list of unique values for each field of the sample characteristics data. Please carefully review the provided information and answer the following questions about this dataset:\n1. Does this dataset contain gene expression data? (Note: Pure miRNA data is not suitable.)\n2. For each of the traits \'Endometrioid Cancer\', \'age\', and \'gender\', please address these points:\n   (1) Is there human data available for this trait?\n   (2) If so, identify the key in the sample characteristics dictionary where unique values of this trait is recorded. 

Understand and verify the answer from GPT, to assign values to the below variables. Assign None to the 'row_id' variables if relevant data row was not found.
Later we need to let GPT format its answer to automatically do these. But given the complexity of this step, let's grow some insight from the free-text answers for now.

In [ ]:
is_gene_availabe = False
trait_row_id = 1
age_row_id = 3
gender_row_id = 5

trait_type = 'binary'

In [ ]:
is_available = is_gene_availabe and (trait_row_id is not None)
if not is_available:
    save_cohort_info(cohort, JSON_PATH, is_available)
    print("This cohort is not usable. Please skip the following steps and jump to the next accession number.")

This cohort is not usable. Please skip the following steps and jump to the next accession number.


In [ ]:
# Verify and use the functions generated by GPT

def convert_trait(value):
    if 'endometrioid adenocarcinoma' in value:
        return '1'  # Presence of endometrioid ovarian cancer
    else:
        return None  # Absence of endometrioid ovarian cancer (or unknown)



def convert_age(value):
    try:
        # Try converting the value to int
        return int(value)
    except (ValueError, TypeError):
        # Return None if conversion is not possible
        return None

def convert_gender(value):
    if value.lower() == 'f':
        return 0
    elif value.lower() == 'm':
        return 1
    else:
        return None


In [ ]:
selected_clinical_data = geo_select_clinical_features(clinical_data, TRAIT, trait_row_id, age_row_id, gender_row_id,
                                                      convert_trait, convert_age, convert_gender)
selected_clinical_data.head()

,GSM2477471,GSM2477472,GSM2477473,GSM2477474,GSM2477475,GSM2477476,GSM2477477,GSM2477478,GSM2477479,GSM2477480,...,GSM2477572,GSM2477573,GSM2477574,GSM2477575,GSM2477576,GSM2477577,GSM2477578,GSM2477579,GSM2477580,GSM2477581
Endometrioid Cancer,1,1,1,1,1,1,1,1,1,1,...,1,1,1,1,1,1,1,1,1,1


### Genetic data preprocessing and final filtering

In [ ]:
genetic_data = get_genetic_data(matrix_file)
genetic_data.head()

,ID,GSM2477471,GSM2477472,GSM2477473,GSM2477474,GSM2477475,GSM2477476,GSM2477477,GSM2477478,GSM2477479,...,GSM2477572,GSM2477573,GSM2477574,GSM2477575,GSM2477576,GSM2477577,GSM2477578,GSM2477579,GSM2477580,GSM2477581
0,1,-0.097131,-0.721129,-0.200969,0.248083,0.133230,1.052330,-0.751642,0.171953,0.161565,...,-0.420415,0.757306,0.559833,-0.065435,0.130097,-0.376034,0.178725,0.069536,-0.458078,-0.439257
4,5,-0.267935,0.346476,0.222593,-0.413383,-0.212756,-0.017118,-0.273706,-0.091366,-1.018550,...,0.304886,0.432069,0.083725,0.387626,-0.647303,-0.345933,-0.215855,0.000870,0.284662,-0.463050
6,7,-0.044643,0.085500,-0.495459,0.205175,-0.103678,-0.342476,-0.138782,-0.196260,0.009514,...,-0.250966,0.216247,0.190309,0.117912,-0.016080,-0.590373,0.074356,-0.079919,0.243825,-0.286126
8,9,-0.017562,0.214008,0.180765,-0.618012,-0.232288,-0.284775,-0.084320,-0.006493,-0.803437,...,0.182867,0.400557,0.323281,-0.290252,-0.590574,-0.409547,-0.366552,-0.641706,0.315821,-1.078810
9,10,0.226628,0.021370,-0.074525,-0.023975,-0.113401,0.246391,-0.369633,0.038554,-0.247618,...,0.013409,-0.099921,0.127118,0.256611,-0.269693,-0.062230,0.378098,0.020310,0.068756,0.648690


In [ ]:
gene_row_ids = genetic_data['ID'][:20].tolist()
gene_row_ids

['1',
 '5',
 '7',
 '9',
 '10',
 '12',
 '16',
 '17',
 '18',
 '19',
 '20',
 '22',
 '24',
 '25',
 '27',
 '29',
 '32',
 '35',
 '43',
 '45']

Check if the gene dataset requires mapping to get the gene symbols corresponding to each data row.

Reference prompt:

In [ ]:
f'''
Below are the row headers of a gene expression dataset in GEO. Based on your biomedical knowledge, are they human gene symbols, or are they some other identifiers that need to be mapped to gene symbols? Your answer should be concluded by starting a new line and strictly following this format:
requires_gene_mapping = (True or False)

Row headers:
{gene_row_ids}
'''

"\nBelow are the row headers of a gene expression dataset in GEO. Based on your biomedical knowledge, are they human gene symbols, or are they some other identifiers that need to be mapped to gene symbols? Your answer should be concluded by starting a new line and strictly following this format:\nrequires_gene_mapping = (True or False)\n\nRow headers:\n['1', '5', '7', '9', '10', '12', '16', '17', '18', '19', '20', '22', '24', '25', '27', '29', '32', '35', '43', '45']\n"

If not required, jump directly to the gene normalization step

In [ ]:
requires_gene_mapping = True

In [ ]:
if requires_gene_mapping:
    gene_annotation = get_gene_annotation(soft_file)[0]
    gene_annotation_summary = preview_df(gene_annotation)
    gene_annotation_summary

Observe the first few cells in the ID column of the gene annotation dataframe, to find the names of columns that store the gene probe IDs and gene symbols respectively.
Reference prompt:

In [ ]:
if requires_gene_mapping:
    print(f'''
    As a biomedical research team, we extracted the gene annotation data from a series in the GEO database, and saved it to a Python dictionary. Please read the dictionary, and decide which key stores the ID of the probe, and which key stores the gene symbols. Please strict follow this format in your answer:
    probe_name_key = key_name1
    gene_name_key = key_name2

    Gene annotation dictionary:
    {gene_annotation_summary}
    ''')


    As a biomedical research team, we extracted the gene annotation data from a series in the GEO database, and saved it to a Python dictionary. Please read the dictionary, and decide which key stores the ID of the probe, and which key stores the gene symbols. Please strict follow this format in your answer:
    probe_name_key = key_name1
    gene_name_key = key_name2

    Gene annotation dictionary:
    {'ID': ['1', '2', '3', '4', '5'], 'MetaRow': ['12', '12', '12', '12', '12'], 'MetaCol': ['4', '4', '4', '4', '4'], 'SubRow': ['28', '27', '26', '25', '24'], 'SubCol': [28.0, 28.0, 28.0, 28.0, 28.0], 'Reporter ID': [334575.0, 333055.0, 331915.0, 330395.0, 328875.0], 'oligo_id': ['H300009761', 'H300009722', 'H300000470', 'H300000646', 'H300004276'], 'oligo_type': ['I', 'I', 'I', 'I', 'I'], 'gene_id': ['ENSG00000182037', 'ENSG00000180563', 'ENSG00000179449', 'ENSG00000177996', 'ENSG00000176539'], 'transcript_count': [1.0, 1.0, 1.0, 1.0, 1.0], 'representative_transcript_id': ['ENST0000031

In [ ]:
if requires_gene_mapping:
    probe_id_key = 'ID'
    gene_symb_key = 'HUGO'
    gene_mapping = get_gene_mapping(gene_annotation, probe_id_key, gene_symb_key)
    genetic_data = apply_gene_mapping(genetic_data, gene_mapping)

In [ ]:
if NORMALIZE_GENE:
    genetic_data = normalize_gene_symbols_in_index(genetic_data)

INFO:biothings.client:querying 1-1000...
INFO:biothings.client:done.
INFO:biothings.client:Finished.
INFO:biothings.client:Pass "returnall=True" to return complete lists of duplicate or missing query terms.
INFO:biothings.client:querying 1-1000...
INFO:biothings.client:done.
INFO:biothings.client:Finished.
INFO:biothings.client:Pass "returnall=True" to return complete lists of duplicate or missing query terms.
INFO:biothings.client:querying 1-1000...
INFO:biothings.client:done.
INFO:biothings.client:Finished.
INFO:biothings.client:Pass "returnall=True" to return complete lists of duplicate or missing query terms.
INFO:biothings.client:querying 1-1000...
INFO:biothings.client:done.
INFO:biothings.client:Finished.
INFO:biothings.client:Pass "returnall=True" to return complete lists of duplicate or missing query terms.
INFO:biothings.client:querying 1-1000...
INFO:biothings.client:done.
INFO:biothings.client:Finished.
INFO:biothings.client:Pass "returnall=True" to return complete lists of

In [ ]:
merged_data = geo_merge_clinical_genetic_data(selected_clinical_data, genetic_data)
# The preprocessing runs through, which means is_available should be True
is_available = True

In [ ]:
print(f"The merged dataset contains {len(merged_data)} samples.")

The merged dataset contains 111 samples.


In [ ]:
is_trait_biased, merged_data = judge_and_remove_biased_features(merged_data, TRAIT, trait_type=trait_type)
is_trait_biased

For the feature 'Endometrioid Cancer', the least common label is '1' with 111 occurrences. This represents 100.00% of the dataset.
The distribution of the feature 'Endometrioid Cancer' in this dataset is severely biased.



True

In [ ]:
save_cohort_info(cohort, JSON_PATH, is_available, is_trait_biased, merged_data)

In [ ]:
merged_data.head()
if not is_trait_biased:
    merged_data.to_csv(os.path.join(OUTPUT_DIR, cohort+'.csv'), index=False)

### 3. Do regression & Cross Validation

In [5]:
best_cohort, ranked_df = filter_and_rank_cohorts(JSON_PATH, 'has_age')
best_cohort

'Xena'

In [6]:
ranked_df.head()

,cohort_id,is_usable,is_available,is_biased,has_age,has_gender,sample_size,note
0,Xena,True,True,False,True,True,187.0,


In [7]:
merged_data = pd.read_csv(os.path.join(OUTPUT_DIR, best_cohort+'.csv'))

In [8]:
merged_data.head()

,Endometrioid Cancer,Age,ARHGEF10L,HIF3A,RNF17,RNF10,RNF11,RNF13,GTF2IP1,REM1,...,SLC7A10,PLA2G2C,TULP2,NPY5R,GNGT2,GNGT1,TULP3,BCL6B,GSTK1,SELP
0,1,59.0,-1.175592,0.864474,-0.531035,0.426928,-0.601278,-1.67421,0.080406,-1.766546,...,2.149614,-0.086682,-0.748878,-0.987117,-0.262433,4.06601,0.333923,-1.501827,0.998505,-2.315833
1,1,54.0,0.215608,-0.178726,7.416765,0.115128,-0.378178,-0.71591,1.207206,-0.991846,...,-2.090786,-0.086682,-0.207478,-1.587117,-0.143633,0.93371,0.174223,-0.877027,0.217905,0.162267
2,1,69.0,-0.764492,-2.620626,-0.531035,-0.360672,-0.411278,-0.38501,0.412606,-2.099246,...,-1.380786,-0.086682,1.194822,-0.407417,-1.269133,0.77641,0.573223,-0.710827,-0.555895,-0.691533
3,1,51.0,-0.907792,-3.403426,-0.531035,-0.019372,-1.538578,-1.41421,0.358106,1.501754,...,-0.637486,1.357518,-0.748878,-1.587117,-1.495533,1.98251,0.406423,-1.374527,0.759805,-1.819733
4,1,86.0,-1.823192,5.243674,-0.531035,-0.365172,-0.476078,-0.72361,0.165706,0.470854,...,6.647914,-0.086682,0.330522,0.758183,0.961667,3.00791,0.208623,0.311373,-1.538495,-0.985533


In [9]:
# If both age and gender features are available, select 'age' as the condition.
condition = 'Age'
# Remove the other condition to prevent interference.
merged_data = merged_data.drop(columns=['Gender'], errors='ignore').astype('float')

In [10]:
X = merged_data.drop(columns=[TRAIT, condition]).values
Y = merged_data[TRAIT].values
Z = merged_data[condition].values

Select the appropriate regression model depending on whether the dataset shows batch effect.

In [11]:
has_batch_effect = detect_batch_effect(X)
has_batch_effect

True

In [12]:
# Select appropriate models based on whether the dataset has batch effect.
# We experiment on two models for each branch. We will decide which one to choose later.

if has_batch_effect:
    model_constructor1 = VariableSelection
    model_params1 = {'modified': True, 'lamda': 3e-4}
    model_constructor2 = VariableSelection
    model_params2 = {'modified': False}
else:
    model_constructor1 = Lasso
    model_params1 = {'alpha': 1.0, 'random_state': 42}
    model_constructor2 = VariableSelection
    model_params2 = {'modified': False}

In [13]:
trait_type = 'binary'  # Remember to set this properly, either 'binary' or 'continuous'
cv_mean1, cv_std1 = cross_validation(X, Y, Z, model_constructor1, model_params1, target_type=trait_type)

alpha for Lasso: 0.0003
alpha for Lasso: 0.0003
alpha for Lasso: 0.0003
alpha for Lasso: 0.0003
alpha for Lasso: 0.0003
The cross-validation accuracy is 60.00% ± 7.33%


In [14]:
cv_mean2, cv_std2 = cross_validation(X, Y, Z, model_constructor2, model_params2, target_type=trait_type)

/usr/local/lib/python3.10/dist-packages/sparse_lmm/helpingMethods.py:49: RuntimeWarning: invalid value encountered in divide
  ts = beta / np.sqrt(var * sigma)
/usr/local/lib/python3.10/dist-packages/sparse_lmm/helpingMethods.py:49: RuntimeWarning: invalid value encountered in divide
  ts = beta / np.sqrt(var * sigma)
/usr/local/lib/python3.10/dist-packages/sparse_lmm/helpingMethods.py:49: RuntimeWarning: invalid value encountered in divide
  ts = beta / np.sqrt(var * sigma)
/usr/local/lib/python3.10/dist-packages/sparse_lmm/helpingMethods.py:49: RuntimeWarning: invalid value encountered in divide
  ts = beta / np.sqrt(var * sigma)
/usr/local/lib/python3.10/dist-packages/sparse_lmm/helpingMethods.py:49: RuntimeWarning: invalid value encountered in divide
  ts = beta / np.sqrt(var * sigma)


The cross-validation accuracy is 77.30% ± 5.57%


In [15]:
normalized_X, _ = normalize_data(X)
normalized_Z, _ = normalize_data(Z)

# Train regression model on the whole dataset to identify significant genes
model1 = ResidualizationRegressor(model_constructor1, model_params1)
model1.fit(normalized_X, Y, normalized_Z)

model2 = ResidualizationRegressor(model_constructor2, model_params2)
model2.fit(normalized_X, Y, normalized_Z)

alpha for Lasso: 0.0003


/usr/local/lib/python3.10/dist-packages/sparse_lmm/helpingMethods.py:49: RuntimeWarning: invalid value encountered in divide
  ts = beta / np.sqrt(var * sigma)


### 5. Discussion and report

In [16]:
feature_cols = merged_data.columns.tolist()
feature_cols.remove(TRAIT)

threshold = 0.05
interpret_result(model1, feature_cols, TRAIT, condition, threshold=threshold, save_output=True,
                 output_dir=OUTPUT_DIR, model_id=1)

Effect of the condition on the target variable:
Variable: Age
Coefficient: 0.0265
Found 59 genes with non-zero coefficients associated with the trait 'Endometrioid Cancer' conditional on the factor 'Age'. These genes are identified as significant based on the regression model.
    Variable  Coefficient  Absolute Coefficient
       IQCF6    -1.959773              1.959773
       OR8D1    -1.895187              1.895187
      OR56A5    -1.746329              1.746329
         OAT     1.705616              1.705616
        ZIC4     1.518328              1.518328
       GPHB5     1.495038              1.495038
     MAGEA9B    -1.480171              1.480171
     LDHAL6B     1.471254              1.471254
         FGG    -1.413843              1.413843
       RGPD8    -1.356230              1.356230
      PRSS48     1.291085              1.291085
          C9    -1.289428              1.289428
      LOXHD1    -1.250149              1.250149
          HP    -1.216444              1.216444
  

/content/drive/MyDrive/Colab Notebooks/utils.py:434: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  gene_regression_df['Absolute Coefficient'] = gene_regression_df['Coefficient'].abs()


In [17]:
interpret_result(model2, feature_cols, TRAIT, condition, threshold=threshold, save_output=True,
                 output_dir=OUTPUT_DIR, model_id=2)

Effect of the condition on the target variable:
Variable: Age
Coefficient: 0.0265
Found 106 significant genes associated with the trait 'Endometrioid Cancer' conditional on the factor 'Age', with corrected p-value < 0.05:
   Variable  Coefficient      p_value  corrected_p_value
       POLQ     0.138491 3.264561e-10           0.000006
   C1orf141    -0.072488 9.862976e-10           0.000009
       EXO1     0.130743 1.538646e-09           0.000009
     CDC25C     0.119733 1.367340e-08           0.000059
       MELK     0.125430 1.021097e-07           0.000354
     CRISP2    -0.054289 1.352958e-07           0.000391
       SKA3     0.109809 1.647334e-07           0.000408
      TROAP     0.120025 3.602311e-07           0.000639
     RAD54L     0.121613 3.401048e-07           0.000639
      UHRF1     0.101329 3.681972e-07           0.000639
      CENPF     0.112566 8.978741e-07           0.001416
       BUB1     0.116262 1.017056e-06           0.001470
      OR8J3     0.031531 2.033630e-06

/content/drive/MyDrive/Colab Notebooks/utils.py:443: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  gene_regression_df.loc[:, 'corrected_p_value'] = corrected_p_values
